In [12]:
import simpy
import random
import statistics
import pandas as pd
import numpy as np


In [13]:
wait_times = []

In [23]:
class House(object):
    
    #switch control for AC
    air_condition = False
    fridge = True
    heat_pump = True
    #consumption of power by appliances on time step
    lights = 0
    
    #power_consumption parameters
    television_power = 10
    microwave_power = 4
    air_condition_power = 30
    fridge_power = 14
    heat_pump_power = 20
    
    #state variables, not sure abt water temp
    water_temp = 30
    water_target = 40
    home_temp = 28
    target_temp = 27
    fridge_temp = 10
    fridge_target = 5
    
    #set up as the duration left that appliance n is running, 0 means it is off
    microwave= 0
    television = 4

    net_load = 0
    power_thresh = 200
    
    def __init__(self, env):
        self.env = env

    def get_event_power(self):

            if self.television != 0:
                #print("TELEVISION "+str(self.television))
                self.television=self.television-1
            #this should be modified to match actual probability distribution
            elif random.randint(1, 20)>19:
                #TODO w prob distribution
                self.television = random.randint(300, 7200)
                print("tv turned on for "+ str(self.television/60)+ " minutes")
            if self.microwave!=0:
                self.microwave-=1
            elif random.randint(1, 5)>4:
                #TODO w prob distribution
                self.microwave = random.randint(30, 500)
                #print("microwave turned on for "+ str(self.microwave/60)+ " minutes")

            return self.television_power+self.microwave_power

    def get_reccuring_power(self):

        load = 0
        if self.air_condition:
            load+= self.air_condition_power
            self.home_temp-=0.0025
            if self.home_temp<self.target_temp:
                self.air_condition = False
        else:
            if self.home_temp>self.target_temp:  
                #print("ac turned on")
                self.air_condition = True
            self.home_temp+=0.0045
            #TODO w prob distribution

            #print("AC turned on")
        if self.fridge:
            load+= self.fridge_power
            self.fridge_temp-=0.0025
            if self.fridge_temp<self.fridge_temp:
                self.fridge = False
        else:
            if self.fridge_temp>self.fridge_target:  
                self.fridge = True
            self.fridge_temp+=0.0045
            #TODO w prob distribution

            #print("Fridge turned on")
        if self.heat_pump:
            load+= self.heat_pump_power
            self.water_temp+=0.025
            if self.water_temp>self.water_target:
                self.heat_pump = False
        else:
            if self.fridge_temp>self.fridge_target:  
                self.heat_pump = True
            self.water_temp-=0.025
            #TODO w prob distribution

            #print("Heat Pump turned on")

        return load

    def get_light_power(self):
        #needs to be matched to lights prob distribution given time of day

        self.lights = random.randint(10, 20)
        return self.lights

    def run_load(self):
        #print("Next")
        self.net_load = 0
        self.net_load+=self.get_event_power()
        self.net_load+=self.get_reccuring_power()
        self.net_load+=self.get_light_power()
        #print("LOAD IS"+str(self.net_load))
        return self.net_load


    


In [18]:
#Measurements / Observations
# 
'''
1 week of simulation with 10 loads takes roughly 22-23 seconds on my computer
1 day with 10 loads takes roughly 3 seconds

With the current buy / sell model (no battery) the price either skews to the left 
or the right, so it's a bit tougher to make highly consistent

AC stays almost consistently on to maintain exact temperature, but if pushed to degree
or a smaller precision power / ac time can be saved significantly

Load modelling seems to be relatively functional. Controls will be maintained for only 
sections of the load modelling, such as AC, Heat Pump, and Fridge
    - based on the AMPdS dataset these are actually a substantial portion of the load
    
Tweaking seemingly does a lot to save on loads

''' 

0


In [ ]:
total_load =  0
        
def run_grid_loads(env, loads):

    global total_load
    total_load= 0
    
    for l in loads:
        total_load+=l.run_load()
    
    return total_load


timestep = 0

def run_simulation(env, num_houses, renewable_sources, num_batteries):
    global timestep
    houses = []
    for i in range(0, num_houses):
        houses.append(House(env))

    while True:
        yield env.timeout(1)  # Wait a bit before generating a new person
        timestep += 1
        run_grid_loads(env, houses)
        #print(get_power(renewable_sources))

ren_power = 0
#replace with probabilistic model sampling
energy_price = 1
total_cost = 0

def get_power(rens):
    global ren_power
    global total_cost
    ren_power = 0
    #would likely get renewables probabilistic cost at this time
    for i in range(0, rens):
        ren_power+=random.randint(220, 290)
    
    if total_load<=ren_power:
        total_cost -= (ren_power-total_load)*energy_price
    else:
        total_cost -= (ren_power-total_load)*energy_price
    
    return total_cost

def main():
    # Setup
    random.seed(42)

    # Run the simulation
    env = simpy.Environment()
    env.process(run_simulation(env, 10, 4, 1))
    #Seconds to run the simulation
    env.run(until=86400)


if __name__ == '__main__':
    main()